In [1]:
import pylangacq as pla

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
import glob

In [5]:
import os

In [6]:
pla.__version__

'0.11.0'

Data grab

In [7]:
cookie_data = pla.read_chat('./Pitt/*/cookie/*.cha')

In [8]:
sentence_data = pla.read_chat('./Pitt/*/sentence/*.cha')

In [9]:
sentence_fnames = glob.glob(os.getcwd()+'/Pitt/*/sentence/*.cha')

In [10]:
sentence_fnames = sorted(sentence_fnames)

In [11]:
# Control, Dementia
cookie_fnames = glob.glob(os.getcwd()+'/Pitt/*/cookie/*.cha')

In [12]:
cookie_fnames = sorted(cookie_fnames)

In [13]:
#samples
len(sentence_fnames+cookie_fnames)

792

In [17]:
len(cookie_data)

552

In [18]:
len(sentence_data)

240

In [14]:
# for random batching/data grabbing 
fnames_dict = {i:fname for (i,fname) in enumerate(sentence_fnames+cookie_fnames)}

Targets (MMSE scores)

In [15]:
# fill missing mmse with group's mean mmse
def get_targets(dataset,filenames):
    
    targets = []
    groups = {}
    
#     for fname in sorted(filenames):
    for fname in filenames:
        # PAR MMSE
#         print(fname)
        target = dataset.headers()[fname]['Participants']['PAR']['education']
        group = dataset.headers()[fname]['Participants']['PAR']['group']
        
        # to be replaced with group mean
        if target is '':
            targets.append(group)
            
        else:
            targets.append(int(target))
            if group in groups.keys():
                groups[group].append(int(target))
            else:
                groups[group] = [int(target)]
                
#     print('targets',targets)            
#     print('groups',groups)
    
    for g in groups.keys():
        m = np.mean(groups[g])
        groups[g] = int(m) if m % 1 > 5 else int(m) + 1
    
    #TODO replace with tensor
    targetseries = pd.Series(targets)
    
#     print('groups_mean',groups)
    
    for g in groups.keys():
        targetseries[[i for i,x in enumerate(targets) if x==g]] = groups[g]

    return targetseries

In [16]:
sentence_data_targets = get_targets(sentence_data,sentence_fnames)

In [17]:
cookie_data_targets = get_targets(cookie_data,cookie_fnames)

In [18]:
len(sentence_data_targets) == len(sentence_data)

True

In [19]:
len(cookie_data_targets) == len(cookie_data)

True

Load addb vocab glove word embeddings 

In [34]:
import torch

In [35]:
glove_path = './PretrainedWordEmb/'

In [36]:
addbvocab = pickle.load(open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'rb'))

In [38]:
len(addbvocab)

2188

---------------------------------------------

BUILD VOCAB

In [20]:
vocabset = set()

In [21]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        tokens = set([tokenraw for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        vocabset.update(tokens)

In [22]:
len(vocabset)

2188

db Vocab as gloVe emb 

In [43]:
sorted(list(addbvocab)) == sorted(list(vocabset))

True

BUILD MOR: POS + GRAMMATICAL CATEGORIES

In [23]:
posset = set()

In [24]:
for corpus,data in [(cookie_fnames,cookie_data),(sentence_fnames,sentence_data)]: 
    for i in range(len(corpus)):
        pos = set([pos for sent in data.tagged_sents(participant='PAR',by_files=True)[corpus[i]] for (tokenraw,pos,tokenstem,dependency) in sent])
        posset.update(pos)

In [26]:
# to concat to embedding tensor 
pos_dict = {i:pos for (i,pos) in enumerate(posset)}

In [25]:
len(posset)

71

--------------------------------------------------------------------------------------------------

VOCAB EMB

In [27]:
#https://github.com/Blosc/bcolz
# import bcolz

In [31]:
# import zipfile

In [32]:
# with zipfile.ZipFile('./PretrainedWordEmb/glove.42B.300d.zip', 'r') as zip_ref:
#     zip_ref.extractall('./PretrainedWordEmb/')

load pretrained glove (common crawl 42B tokens, 1.9M vocab, uncased, 300d vectors)

In [52]:
# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         break
    

In [58]:
# words = []
# idx = 0
# word2idx = {}
# vectors = bcolz.carray(np.zeros(1), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')

# with open(f'{glove_path}/glove.42B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(np.float)
#         vectors.append(vect)

In [29]:
import pickle

In [30]:
# vectors = bcolz.carray(vectors[1:].reshape((-1,300)), rootdir=f'{glove_path}/glove.42B.300.dat', mode='w')
# vectors.flush()

In [31]:
# pickle.dump(words, open(f'{glove_path}/glove.42B.300_words.pkl', 'wb'))
# pickle.dump(word2idx, open(f'{glove_path}/glove.42B.300_idx.pkl', 'wb'))

In [70]:
# vectors = bcolz.open(f'{glove_path}/glove.42B.300.dat')[:]

In [32]:
# words = pickle.load(open(f'{glove_path}/glove.42B.300_words.pkl', 'rb'))
# word2idx = pickle.load(open(f'{glove_path}/glove.42B.300_idx.pkl', 'rb'))

In [ ]:
# UNK token -- average of embeddings
# average_vec = np.mean(vectors, axis=0)
# print(average_vec)

In [180]:
'UNK' in words

False

In [184]:
type(words)

list

In [188]:
type(word2idx)

dict

In [189]:
words.append('UNK')

In [212]:
vectors.shape

(1917494, 300)

In [244]:
vectors = np.concatenate((vectors,np.reshape(average_vec,(1,300))))

In [252]:
len(vectors)

1917495

In [253]:
word2idx['UNK'] = len(vectors) - 1

In [200]:
len(words)

1917495

In [254]:
import string
from nltk.tokenize.treebank import TreebankWordTokenizer

def get_glovembs(sample):
    
    def handle_contractions(x):
        tokenizer = TreebankWordTokenizer()
        x = tokenizer.tokenize(x)
        x = ' '.join(x)
        return x
    
    # preprocess acc to pretrained embeddings 
    # lower
    # replace _, -
    # rm contractions
    # rm nonascii chars
    
    sample = [s for s in sample]
    
    preprocessample = [handle_contractions(t).split()[0] for t in [token.lower().replace('_','-') for token in sample]]
    
    preprocessample = [''.join([i if ord(i) < 128 else '' for i in w]) for w in preprocessample]
    # {w: vectors[word2idx[w]] for w in words}
    return {sample[i]: torch.tensor(vectors[word2idx[preprocessample[i]]]) if preprocessample[i] in words else torch.tensor(vectors[word2idx['UNK']]) for i in range(len(sample))}

In [255]:
addb_vocab_embs = get_glovembs(vocabset)

In [257]:
len(addb_vocab_embs)

2188

In [272]:
addb_vocab_embs['UNK']

KeyError: 'UNK'

In [277]:
# replace CLITIC emb with UNK token (mistaken for word 'clitic')
addb_vocab_embs['CLITIC'] = vectors[word2idx['UNK']]

In [279]:
pickle.dump(addb_vocab_embs, open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'wb'))

In [280]:
addbvocab = pickle.load(open(f'{glove_path}/addb.vocab.glove.42B.300_words.pkl', 'rb'))

In [282]:
len(addbvocab)

2188

In [283]:
list(addbvocab) == list(addb_vocab_embs)

True

In [152]:
len(vocabset_proc)

2188

In [154]:
# list generated from preprocessing steps subsumed into get_glovembs()
len(unks)

26

In [ ]:
# ignore CLITIC

In [183]:
unks

['cookeiejar',
 "i-don't-know",
 'cappdf',
 '//',
 '',
 '//',
 'alrightie',
 '..',
 'oh-boy',
 'hmhunh',
 'sewickly',
 'tazmania-dutch',
 'doctor-dick',
 'how-about',
 'god-bless-you',
 '',
 'oh-dear',
 'how-come',
 'i-mean',
 'what-about',
 'joyce-kilmer',
 'the-window-of-the-xxx',
 'dave-branton',
 'lots-of',
 'a-lot-of',
 'lee-a']